In [8]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.4934, 0.0670, 0.7651],
        [0.9455, 0.0414, 0.1481],
        [0.4420, 0.6143, 0.5426],
        [0.2448, 0.7805, 0.4293],
        [0.3445, 0.8043, 0.5269]])


In [9]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [10]:
df = pd.read_pickle('processed_data/SARIMA_model_data')
print(df.shape)

n = df.shape[0]

(35064, 40)


In [11]:
df.head(10)

Load1  Temp1     Load2  Temp2     Load3  Temp3  Load4  Temp4   Load5  \
0  16853.0   38.0  126259.0   42.0  136233.0   42.0  484.0   42.0  6829.0   
1  16450.0   36.0  123313.0   42.0  133055.0   42.0  457.0   42.0  6596.0   
2  16517.0   35.0  119192.0   40.0  128608.0   40.0  450.0   40.0  6525.0   
3  16873.0   30.0  117507.0   39.0  126791.0   39.0  448.0   39.0  6654.0   
4  17064.0   30.0  118343.0   40.0  127692.0   40.0  444.0   40.0  6977.0   
5  17727.0   29.0  121228.0   41.0  130805.0   41.0  490.0   41.0  7330.0   
6  18574.0   28.0  126731.0   38.0  136743.0   38.0  523.0   38.0  7795.0   
7  19355.0   28.0  132289.0   33.0  142741.0   33.0  581.0   33.0  8303.0   
8  19534.0   28.0  139941.0   36.0  150997.0   36.0  612.0   36.0  9190.0   
9  18611.0   33.0  141950.0   43.0  153165.0   43.0  633.0   43.0  9516.0   

   Temp5  ...   Load16  Temp16   Load17  Temp17    Load18  Temp18   Load19  \
0   42.0  ...  28752.0    42.0  30645.0    42.0  200946.0    42.0  82298.0   
1   42.0  ...  27851.0    42.0  30461.0    42.0  195835.0    42.0  79827.0   
2   40.0  ...  27631.0    40.0  30197.0    40.0  194093.0    40.0  77728.0   
3   39.0  ...  27986.0    39.0  30264.0    39.0  194708.0    39.0  76433.0   
4   40.0  ...  29160.0    40.0  30907.0    40.0  202458.0    40.0  78172.0   
5   41.0  ...  29226.0    41.0  31617.0    41.0  206923.0    41.0  77871.0   
6   38.0  ...  31411.0    38.0  33220.0    38.0  219789.0    38.0  82895.0   
7   33.0  ...  33447.0    33.0  34531.0    33.0  230457.0    33.0  87599.0   
8   36.0  ...  34591.0    36.0  34837.0    36.0  240851.0    36.0  90546.0   
9   43.0  ...  33637.0    43.0  33418.0    43.0  243584.0    43.0  86826.0   

   Temp19   Load20  Temp20  
0    45.0  79830.0    42.0  
1    43.0  77429.0    42.0  
2    41.0  75558.0    40.0  
3    37.0  75709.0    39.0  
4    33.0  77475.0    40.0  
5    32.0  77854.0    41.0  
6    30.0  82571.0    38.0  
7    29.0  85996.0    33.0  
8    30.0  91065.0    36.0  
9    41.0  94689.0    43.0  

[10 rows x 40 columns]

In [18]:
training_set = df.Load1.to_numpy().reshape(-1,1)
print(training_set.shape)

(35064, 1)


In [38]:
def seven_day_to_one_day(data):
    
    INPUT_DAYS = 7
    OUTPUT_DAYS = 1
    
    num_periods = int(data.shape[0] / 24) - INPUT_DAYS
    assert(data.shape[0] % 24 == 0)
    
    input_length = INPUT_DAYS * 24
    output_length = OUTPUT_DAYS * 24
    
    x = np.zeros((num_periods, input_length))
    y = np.zeros((num_periods, output_length))
    
    for period in range(num_periods):
        start_index = period * 24
        _x = data[start_index : start_index + input_length]
        _y = data[start_index + input_length : start_index + input_length + output_length]
        
        x[period,:] = _x.T
        y[period,:] = _y.T
        
    return x, y

# First scale input data to be between [0,1]
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

# Get processed data on which to train
x, y = seven_day_to_one_day(training_data)


train_size = int(len(y) * 0.67)
test_size = len(y) - train_size
dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

# TRAINING
trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

# TESTING
testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [44]:
print(trainX.shape)
print(trainY.shape)

torch.Size([974, 168])
torch.Size([974, 24])


In [45]:
# # For transformer:
# trainX_transformer = torch.unsqueeze(trainX, 2)
# trainY_transformer = torch.unsqueeze(trainY, 2)

# # N x length x num_features
# print(trainX_transformer.shape)
# print(trainY_transformer.shape)

In [ ]:
# class Transformer(nn.Module):

#     def __init__(self, num_features):
#         super(LSTM, self).__init__()
        
#         # Batch first means that input is shape (N, S, E) : (batch, seq, feature)
#         self.transformer = nn.Transformer(d_model = num_features, batch_first=True) # Just use default settings

#     def forward(self, x):
        
#         out = self.transformer(x)
        
#         return out

In [ ]:
class SelfAttn(nn.Module):
    def __init__(self, source_seq_length, target_seq_length, embed_dim, num_heads):
        # For Pytorch:
        super(Transformer, self).__init__()
        
        self.source_seq_length = source_seq_length
        self.target_seq_length = target_seq_length
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, need_weights=False)
        
        self.query = nn.Linear(source_seq_length, embed_dim)
        self.key = nn.Linear(source_seq_length, embed_dim)
        self.value = nn.Linear(source_seq_length, embed_dim)
        
        self.output = nn.Linear(embed_dim, target_seq_length)
        
    def forward(self,x):
        
        # Self attention. Model learns its own query, key and value via Linear layers
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
        
        attn_output = self.multihead_attn(query, key, value)
        
        # Make it the right shape
        out = self.output(attn_output)
        
        return out

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out